<div align="right" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/Logo blue_dark.png"  style="width:25px" align="right";/>
</div>

# Intro to the Second Normal Form – 2NF
© ExploreAI Academy

In this train, we will learn how to normalise a database table into the Second Normal Form (2NF) by first ensuring that the table is in the First Normal Form (1NF) and then creating separate tables for related entities. We will also be introduced to the concept of a junction table. 

> ⚠️ This exercise extends the concepts introduced in the previous one, **Intro to the First Normal Form – 1NF**. Therefore, it's important to ensure that you continue using the modified `dam_levels.db` database after successfully completing the previous exercise.
 
> ⚠️ Since the queries here will modify the database, you will have to get a fresh copy to redo the code cells.

## Learning objectives

By the end of this train, you should:
- Understand the key principles of achieving Second Normal Form (2NF) in database normalisation.
- Practise splitting a denormalised table into separate tables, addressing partial functional dependencies.
- Master the concept of a junction table and its role in linking data between two related tables in a database schema.

## 1. Overview
To get the table in 2NF, we need to ensure the following:

- The table is in 1NF.
- All partial functional dependencies should be removed – all non-key columns should depend on the key column, and anything else should be placed in a separate table. 

<div>
Let's look at normalising the 1NF dams database containing information about Cape Town dam water levels leading up to the 2018 Cape Town water crisis.
</div>

## Connecting to the database

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook. 
# If you get an error here, make sure that mysql and pymysql are installed correctly. 

%load_ext sql

In [2]:
# Load SQLite database
%sql sqlite:///../db/dam_levels.db

Before we proceed, let's first take a closer look at the `dam_levels_1nf` table:

In [6]:
%%sql

SELECT *
FROM dam_levels_1nf LIMIT 10;

 * sqlite:///../db/dam_levels.db
Done.


AssessmentId,year,dam_name,Assessment_Officer,Officer_Reg,water_level,dam_latitude,dam_longitude
1,2012,WEMMERSHOEK,P. M. Naidoo,201013,48.2,-33.826246,19.091828
2,2012,VOËLVLEI,A. T. Sijovu,202256,15,-33.34178,19.04105
3,2012,HELY-HUTCHINSON,P. M. Naidoo,201013,14.2,-33.976929,18.409568
4,2012,WOODHEAD,A. T. Sijovu,202256,35.3,-33.977341,18.404046
5,2013,WEMMERSHOEK,P. M. Naidoo,201013,53.3,-33.826246,19.091828
6,2013,VOËLVLEI,A. T. Sijovu,202256,16.6,-33.34178,19.04105
7,2013,HELY-HUTCHINSON,P. M. Naidoo,201013,15.2,-33.976929,18.409568
8,2013,WOODHEAD,A. T. Sijovu,202256,35.9,-33.977341,18.404046
9,2015,WEMMERSHOEK,P. M. Naidoo,201013,47.5,-33.826246,19.091828
10,2015,VOËLVLEI,A. T. Sijovu,202256,11.9,-33.34178,19.04105


Looking at the table, we notice two distinct entities: assessments and dam information.


As such, we need to create two tables:

- `assessments` – with columns: `AssessmentId`, `year`, `Assessment_Officer`, `Officer_Reg`, `water_level`
- `dams` – with columns: `dam_name`, `dam_latitude`, `dam_longitude`

## Exercise


### Exercise 1

Create the `assessments` table.

In [7]:
%%sql 

CREATE TABLE assessments (
    AssessmentId INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER,
    Assessment_Officer VARCHAR(100),
    Officer_Reg INTEGER,
    water_level NUMERIC(10,1)
);

 * sqlite:///../db/dam_levels.db
Done.


[]

### Exercise 2

Copy the relevant data into the `assessments` table from `dam_levels_1nf`.

In [8]:
%%sql

INSERT INTO 
    assessments(
            AssessmentId, 
            year, 
            Assessment_Officer,  
            Officer_Reg, 
            water_level
        )
SELECT 
    AssessmentId, 
    year, 
    Assessment_Officer, 
    Officer_Reg, 
    water_level
FROM 
    dam_levels_1nf
ORDER BY year;

 * sqlite:///../db/dam_levels.db
18 rows affected.


[]

### Exercise 3

Create the `dams` table.

In [13]:
%%sql

CREATE TABLE dams (
    dam_name VARCHAR(100),
    dam_latitude NUMERIC(3,6),
    dam_longitude NUMERIC(3,6)
);

 * sqlite:///../db/dam_levels.db
Done.


[]

### Exercise 4

Copy the relevant data into the `dams` table from `dam_levels_1nf`.

In [14]:
%%sql

INSERT INTO 
    dams(
            dam_name, 
            dam_latitude, 
            dam_longitude
        )
SELECT
    dam_name, 
    dam_latitude, 
    dam_longitude
FROM 
    dam_levels_1nf
GROUP BY dam_name -- Because there are duplicates

 * sqlite:///../db/dam_levels.db
6 rows affected.


[]

### Exercise 5

At this point, we've created two tables: the `assessments` and the `dams` tables. However, the assessments and dam information are no longer linked, i.e. we don't know which assessment belongs to which dam. As such, we'll need to create a junction table (i.e. a linking table, in this case, a table that connects an assessment to a dam).

To make a valid junction table, we'll need primary keys from both tables.

Using the `AssessmentId` from the `assessments` table and the `dam_name` from the `dams` table, create the junction table. Call it `dam_assessments`.

In [17]:
%%sql 

CREATE TABLE 
    dam_assessments (
        AssessmentId INTEGER PRIMARY KEY AUTOINCREMENT,
        dam_name VARCHAR(100)
    );

 * sqlite:///../db/dam_levels.db
Done.


[]

### Exercise 6

Insert the relevant data into the `dam_assessments` table.

In [18]:
%%sql
INSERT INTO dam_assessments
SELECT AssessmentId, dam_name FROM dam_levels_1nf

 * sqlite:///../db/dam_levels.db
18 rows affected.


[]

### Exercise 7

Delete the now redundant `dam_levels_1nf` table.

In [19]:
%sql DROP TABLE IF EXISTS dam_levels_1nf

 * sqlite:///../db/dam_levels.db
Done.


[]

## Solutions

### Exercise 1

In [ ]:
%%sql 

CREATE TABLE assessments (
    AssessmentId INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER,
    Assessment_Officer VARCHAR(100),
    Officer_Reg INTEGER,
    water_level NUMERIC(10,1)
);

### Exercise 2



In [ ]:
%%sql

INSERT INTO 
    assessments(
            AssessmentId, 
            year, 
            Assessment_Officer,  
            Officer_Reg, 
            water_level
        )
SELECT 
    AssessmentId, 
    year, 
    Assessment_Officer, 
    Officer_Reg, 
    water_level
FROM 
    dam_levels_1nf
ORDER BY year;

### Exercise 3



In [ ]:
%%sql

CREATE TABLE dams (
    dam_name VARCHAR(100),
    dam_latitude NUMERIC(3,6),
    dam_longitude NUMERIC(3,6)
);

### Exercise 4

In [ ]:
%%sql

INSERT INTO 
    dams(
            dam_name, 
            dam_latitude, 
            dam_longitude
        )
SELECT 
    dam_name, 
    dam_latitude, 
    dam_longitude
FROM 
    dam_levels_1nf
GROUP BY dam_name;

### Exercise 5

In [ ]:
%%sql 

CREATE TABLE 
    dam_assessments (
        AssessmentId INTEGER PRIMARY KEY AUTOINCREMENT,
        dam_name VARCHAR(100)
    );

### Exercise 6

In [ ]:
%%sql

INSERT INTO 
    dam_assessments(
            AssessmentId,
            dam_name
        )
VALUES 
    (1 	, "WEMMERSHOEK"), 
    (2 	, "VOËLVLEI"),
    (3 	, "HELY-HUTCHINSON"),
    (4 	, "WOODHEAD"), 
    (5 	, "STEENBRAS LOWER"), 
    (6 	, "STEENBRAS UPPER"), 
    (7 	, "WEMMERSHOEK"), 
    (8 	, "VOËLVLEI"), 
    (9 	, "HELY-HUTCHINSON"), 
    (10 , "WOODHEAD"), 
    (11 , "STEENBRAS LOWER"), 
    (12 , "STEENBRAS UPPER"), 
    (13 , "WEMMERSHOEK"), 
    (14 , "VOËLVLEI"), 
    (15 , "HELY-HUTCHINSON"),
    (16 , "WOODHEAD"), 
    (17 , "STEENBRAS LOWER"), 
    (18 , "STEENBRAS UPPER");

### Exercise 7

In [ ]:
%%sql

DROP TABLE dam_levels_1nf;

## Summary

At this point, we have split the dam_levels_1nf table into the following tables:

|name|
---
assessments
dam_assessments
dams

Since these tables are in 1NF and do not have any partial functional dependencies, we have successfully normalised our tables into the Second Normal Form!

**Important**: After completing this exercise, ensure that you save the `dam_levels.db` file, as it will be used as a foundation for the next exercise.

#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>